# Day 45

In anticipation of Week 14's upcoming games, I wanted to calculate the best and worst teams to go up against by position. In other words, which teams allow the most & least fantasy points by position through Week 13 of the 2022 season?  

I'll solve this question by using mainly SQL.

In [15]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette('deep')

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

In [16]:
query = """
WITH fantasy_totals_by_team AS (
    -- Get the total fantasy points scored per team, per position, per week through Week 13
    SELECT
        recent_team,
        position,
        week,
        ROUND(SUM(fantasy_points), 2) as tot_pts,
        ROUND(SUM(fantasy_points_ppr), 2) as tot_pts_ppr
    FROM weekly
    WHERE season = 2022
        AND week <= 13
        AND position in ('QB', 'RB', 'WR', 'TE')
    GROUP BY recent_team, position, week),
-- Get schedule info through week 13
    home_games AS (
        SELECT 
            game_id,
            season,
            week, 
            home_team AS team,
            away_team AS opp_team
        FROM schedules
        WHERE season = 2022
            AND week <= 13
    ),
    away_games AS (
       SELECT 
            game_id,
            season,
            week, 
            away_team AS team,
            home_team AS opp_team
        FROM schedules
        WHERE season = 2022
            AND week <= 13
    ),
    stacked AS (
        SELECT *
        FROM home_games
        UNION
        SELECT *
        FROM away_games
    ),
-- Join the two temp datasets
-- Every team will have it's total fantasy points per team appended to it
joined_data AS (
    SELECT
        game_id,
        season,
        stacked.week,
        team,
        opp_team,
        position,
        tot_pts,
        tot_pts_ppr
    FROM stacked
    LEFT JOIN fantasy_totals_by_team
        ON fantasy_totals_by_team.recent_team = stacked.team
        AND fantasy_totals_by_team.week = stacked.week),
-- Only use opp_team to get the points scored scored against
total_pts_scored_against AS (
    SELECT
        opp_team,
        position,
        SUM(tot_pts) AS tot_pts_scored_against,
        SUM(tot_pts_ppr) AS tot_pts_ppr_scored_against
    FROM joined_data
    GROUP BY opp_team, position),
-- Utilize RANK() to get the season-level rankings
season_rankings AS (
    SELECT
        opp_team AS team,
        position,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_scored_against DESC) AS r_pts_scored_against,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr_scored_against DESC) AS r_pts_ppr_scored_against
    FROM total_pts_scored_against
    ORDER BY position, r_pts_ppr_scored_against)
SELECT *
FROM season_rankings
"""

df = pd.read_sql(query, conn)
df

,team,position,r_pts_scored_against,r_pts_ppr_scored_against
0,DET,QB,1,1
1,LV,QB,2,2
2,ARI,QB,3,3
3,TEN,QB,4,4
4,MIA,QB,5,5
...,...,...,...,...
123,IND,WR,30,28
124,HOU,WR,27,29
125,CIN,WR,29,30
126,NYJ,WR,31,31


## Standard Scoring

In [21]:
standard = df.drop('r_pts_ppr_scored_against', axis=1).sort_values('r_pts_scored_against')


for pos in ['QB', 'RB', 'WR', 'TE']:
    print(standard.query(f"(position == '{pos}') & (r_pts_scored_against <= 3 | r_pts_scored_against >= 30)"))

   team position  r_pts_scored_against
0   DET       QB                     1
1    LV       QB                     2
2   ARI       QB                     3
29  DEN       QB                    30
30  HOU       QB                    31
31  PHI       QB                    32
   team position  r_pts_scored_against
33  HOU       RB                     1
32  CHI       RB                     2
36   GB       RB                     3
62   NE       RB                    30
59  TEN       RB                    31
63   SF       RB                    32
    team position  r_pts_scored_against
96   TEN       WR                     1
99   PIT       WR                     2
97   ATL       WR                     3
123  IND       WR                    30
126  NYJ       WR                    31
127  DEN       WR                    32
   team position  r_pts_scored_against
65  SEA       TE                     1
64  ARI       TE                     2
69  DET       TE                     3
95   SF       TE  

## PPR Scoring

In [22]:
ppr = df.drop('r_pts_scored_against', axis=1).sort_values('r_pts_ppr_scored_against')


for pos in ['QB', 'RB', 'WR', 'TE']:
    print(ppr.query(f"(position == '{pos}') & (r_pts_ppr_scored_against <= 3 | r_pts_ppr_scored_against >= 30)"))

   team position  r_pts_ppr_scored_against
0   DET       QB                         1
1    LV       QB                         2
2   ARI       QB                         3
29  DEN       QB                        30
30  HOU       QB                        31
31  PHI       QB                        32
   team position  r_pts_ppr_scored_against
32  CHI       RB                         1
33  HOU       RB                         2
34  IND       RB                         3
61   TB       RB                        30
62   NE       RB                        31
63   SF       RB                        32
    team position  r_pts_ppr_scored_against
96   TEN       WR                         1
97   ATL       WR                         2
98   MIN       WR                         3
125  CIN       WR                        30
126  NYJ       WR                        31
127  DEN       WR                        32
   team position  r_pts_ppr_scored_against
64  ARI       TE                         1
65  